In [ ]:

bucket_name = 'hawaii-marine-debris'
file_path_template = "{}/jpg/{}"
downloaded_tile = set()
csv_rows = []
islands= ['niihau']#'lanai','bigisland',''
local_tmp_folder = "/home/paperspace/data"
local_tmp_image_folder = "/home/paperspace/data/niihau/images/{filename}"

tile_with_debris = []


In [ ]:
import os
import boto3 

import boto3 
boto3.setup_default_session(profile_name='hawaii')
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
version='v2'

s3_resource.Bucket(bucket_name).download_file("niihau/deduped_annotations.csv", "/tmp/niihau/annotations.csv")
def download_image_from_s3(s3_key, island='niihau'):
    basename = os.path.basename(s3_key)
    local_file_path = local_tmp_image_folder.format(island=island, filename=basename)
    s3_resource.Bucket(bucket_name).download_file(s3_key, local_file_path)
    return local_file_path

In [ ]:
import pandas as pd
df  =pd.read_csv("/tmp/niihau/annotations.csv")


In [ ]:
df.groupby('label').count()

In [ ]:
def local_file_path(s3_key, island='niihau'):
    basename = os.path.basename(s3_key)
    local_file_path = local_tmp_image_folder.format(island=island, filename=basename)
    return local_file_path
df['local_file_path'] = df['s3_key'].apply(local_file_path)

In [ ]:
import numpy as np
msk = np.random.rand(len(df)) < 0.9

train = df[msk]
test = df[~msk]

train[['local_file_path','x0','y0','x1','y1','label']].to_csv('keras_annotations_train.csv', encoding='utf-8', header=False, index=False)
test[['local_file_path','x0','y0','x1','y1','label']].to_csv('keras_annotations_test.csv', encoding='utf-8', header=False, index=False)


In [ ]:

for index, row in df[:20].iterrows():
    local_file = download_image_from_s3(row['s3_key'])
    print(local_file)